<a href="https://colab.research.google.com/github/Marlucben/Challenge-ONE-Data-Science-Telecom-X-parte-2/blob/main/Challenge_parte_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 🛠️ Preparación de los Datos

In [11]:
import pandas as pd
import numpy as np
url = 'https://raw.githubusercontent.com/Marlucben/Challenge-ONE-Data-Science-Telecom-X-parte-2/refs/heads/main/df_nuevo.json'
df_nuevo = pd.read_json(url , orient="records", lines=True)
df_nuevo.head()

,customerID,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Charges.Days
0,0002-ORFBO,NO,FEMALE,0,YES,YES,9,YES,NO,ONE YEAR,YES,MAILED CHECK,65.6,593.30,2.2
1,0003-MKNFE,NO,MALE,0,NO,NO,9,YES,YES,MONTH-TO-MONTH,NO,MAILED CHECK,59.9,542.40,2.0
2,0004-TLHLJ,YES,MALE,0,NO,NO,4,YES,NO,MONTH-TO-MONTH,YES,ELECTRONIC CHECK,73.9,280.85,2.5
3,0011-IGKFF,YES,MALE,1,YES,NO,13,YES,NO,MONTH-TO-MONTH,YES,ELECTRONIC CHECK,98.0,1237.85,3.3
4,0013-EXCHZ,YES,FEMALE,1,YES,NO,3,YES,NO,MONTH-TO-MONTH,YES,MAILED CHECK,83.9,267.40,2.8


## Elimina columnas que no aportan valor al análisis o a los modelos predictivos, como identificadores únicos (por ejemplo, el ID del cliente). Estas columnas no ayudan en la predicción de la cancelación y pueden incluso perjudicar el desempeño de los modelos.

In [12]:
df_nuevo = df_nuevo.drop(columns=["customerID"])
df_nuevo.head()

,Churn,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,Contract,PaperlessBilling,PaymentMethod,Charges.Monthly,Charges.Total,Charges.Days
0,NO,FEMALE,0,YES,YES,9,YES,NO,ONE YEAR,YES,MAILED CHECK,65.6,593.30,2.2
1,NO,MALE,0,NO,NO,9,YES,YES,MONTH-TO-MONTH,NO,MAILED CHECK,59.9,542.40,2.0
2,YES,MALE,0,NO,NO,4,YES,NO,MONTH-TO-MONTH,YES,ELECTRONIC CHECK,73.9,280.85,2.5
3,YES,MALE,1,YES,NO,13,YES,NO,MONTH-TO-MONTH,YES,ELECTRONIC CHECK,98.0,1237.85,3.3
4,YES,FEMALE,1,YES,NO,3,YES,NO,MONTH-TO-MONTH,YES,MAILED CHECK,83.9,267.40,2.8


## Transforma las variables categóricas a formato numérico para hacerlas compatibles con los algoritmos de machine learning. Utiliza un método de codificación adecuado, como one-hot encoding.

🔎 Sugerencia:
Puedes consultar este artículo para entender mejor cuándo usar get_dummies o OneHotEncoder:

In [13]:
df_nuevo_compatible_ml = pd.get_dummies(df_nuevo, drop_first=True)
df_nuevo_compatible_ml.head()

,SeniorCitizen,tenure,Charges.Monthly,Charges.Total,Charges.Days,Churn_YES,gender_MALE,Partner_YES,Dependents_YES,PhoneService_YES,MultipleLines_NO PHONE SERVICE,MultipleLines_YES,Contract_ONE YEAR,Contract_TWO YEAR,PaperlessBilling_YES,PaymentMethod_CREDIT CARD (AUTOMATIC),PaymentMethod_ELECTRONIC CHECK,PaymentMethod_MAILED CHECK
0,0,9,65.6,593.30,2.2,False,False,True,True,True,False,False,True,False,True,False,False,True
1,0,9,59.9,542.40,2.0,False,True,False,False,True,False,True,False,False,False,False,False,True
2,0,4,73.9,280.85,2.5,True,True,False,False,True,False,False,False,False,True,False,True,False
3,1,13,98.0,1237.85,3.3,True,True,True,False,True,False,False,False,False,True,False,True,False
4,1,3,83.9,267.40,2.8,True,False,True,False,True,False,False,False,False,True,False,False,True


## Calcula la proporción de clientes que cancelaron en relación con los que permanecieron activos. Evalúa si existe un desbalance entre las clases, ya que esto puede impactar en los modelos predictivos y en el análisis de los resultados.

🔎 Sugerencia:
Puedes usar value_counts() de pandas para obtener esta proporción:
Documentación oficial de value_counts()

In [14]:
clientes_que_cancelaron = df_nuevo["Churn"].value_counts(normalize=True)
print(clientes_que_cancelaron)

Churn
NO     0.74281
YES    0.25719
Name: proportion, dtype: float64


##Si deseas profundizar en el análisis, aplica técnicas de balanceo como undersampling o oversampling. En situaciones de fuerte desbalanceo, herramientas como SMOTE pueden ser útiles para generar ejemplos sintéticos de la clase minoritaria.

🔎 Sugerencia:
Puedes leer más sobre cómo manejar el desbalanceo de clases en este artículo:
Artículo en Alura sobre desbalanceo de datos

Evalúa la necesidad de normalizar o estandarizar los datos, según los modelos que se aplicarán. Modelos basados en distancia, como KNN, SVM, Regresión Logística y Redes Neuronales, requieren este preprocesamiento. Por otro lado, modelos basados en árboles, como Decision Tree, Random Forest y XGBoost, no son sensibles a la escala de los datos.

🔎 Sugerencia:
Puedes leer más sobre normalización y estandarización de datos en este artículo:
Artículo en Medium sobre normalización y estandarización en Machine Learning

# 🎯 Correlación y Selección de Variables

Visualiza la matriz de correlación para identificar relaciones entre las variables numéricas. Presta especial atención a las variables que muestran una mayor correlación con la cancelación, ya que estas pueden ser fuertes candidatas para el modelo predictivo.

Investiga cómo variables específicas se relacionan con la cancelación, tales como:

Tiempo de contrato × Cancelación

Gasto total × Cancelación

Utiliza gráficos como boxplots o scatter plots para visualizar patrones y posibles tendencias.

# 🤖 Modelado Predictivo

Divide el conjunto de datos en entrenamiento y prueba para evaluar el rendimiento del modelo. Una división común es 70% para entrenamiento y 30% para prueba, o 80/20, dependiendo del tamaño de la base de datos.

Divide el conjunto de datos en entrenamiento y prueba para evaluar el rendimiento del modelo. Una división común es 70% para entrenamiento y 30% para prueba, o 80/20, dependiendo del tamaño de la base de datos.

Descripción
Editar
Crea al menos dos modelos diferentes para predecir la cancelación de clientes.

Un modelo puede requerir normalización, como Regresión Logística o KNN.

El otro modelo puede no requerir normalización, como Árbol de Decisión o Random Forest.

💡 La decisión de aplicar o no la normalización depende de los modelos seleccionados. Ambos modelos pueden ser creados sin normalización, pero también es una opción combinar modelos con y sin normalización.

Justificación:

Regresión Logística / KNN: Estos modelos son sensibles a la escala de los datos, por lo que la normalización es importante para que los coeficientes o las distancias se calculen correctamente.

Árbol de Decisión / Random Forest: Estos modelos no dependen de la escala de los datos, por lo que no es necesario aplicar normalización.

Si decides normalizar los datos, deberías explicar cómo esta etapa asegura que los modelos basados en distancia o en optimización de parámetros no se vean sesgados por la magnitud de las variables.

Comentarios y Actividad
Mostrar detalles
Escribe un comentario...




Evalúa cada modelo utilizando las siguientes métricas:

Exactitud (Acurácia)

Precisión

Recall

F1-score

Matriz de confusión

Después, realiza un análisis crítico y compara los modelos:

¿Cuál modelo tuvo el mejor desempeño?

¿Algún modelo presentó overfitting o underfitting? Si es así, considera las posibles causas y ajustes:

Overfitting: Cuando el modelo aprende demasiado sobre los datos de entrenamiento, perdiendo la capacidad de generalizar a nuevos datos. Considera reducir la complejidad del modelo o aumentar los datos de entrenamiento.

Underfitting: Cuando el modelo no captura bien las tendencias de los datos, lo que indica que es demasiado simple. Intenta aumentar la complejidad del modelo o ajustar sus parámetros.

Después de elegir los modelos, realiza el análisis de las variables más relevantes para la predicción de la cancelación:

Regresión Logística: Investiga los coeficientes de las variables, que muestran su contribución a la predicción de cancelación.

KNN (K-Nearest Neighbors): Observa cómo los vecinos más cercanos influyen en la decisión de clasificación. Las variables más impactantes pueden ser aquellas que más contribuyen a la proximidad entre los puntos de datos.

Random Forest: Utiliza la importancia de las variables proporcionada por el modelo. Random Forest calcula la importancia basándose en cómo cada variable contribuye a la reducción de la impureza durante las divisiones de los árboles.

SVM (Support Vector Machine): En el SVM, las variables más relevantes son aquellas que influyen en la frontera de decisión entre las clases. Puedes analizar los coeficientes de los vectores de soporte para entender qué variables tienen mayor impacto.

Otros Modelos: Dependiendo del modelo elegido, considera el análisis de métricas específicas para comprender la relevancia de las variables. Por ejemplo, coeficientes en modelos lineales, pesos en redes neuronales, o la importancia relativa en técnicas de boosting (como XGBoost).

# Conclusión

Descripción
Editar
Elaboren un informe detallado, destacando los factores que más influyen en la cancelación, basándose en las variables seleccionadas y en el rendimiento de cada modelo.

Identifiquen los principales factores que afectan la cancelación de clientes y propongan estrategias de retención basadas en los resultados obtenidos.